In [139]:
import pandas as pd
import numpy as np
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, BatchNormalization, Dropout
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier

In [159]:
def main():
    data = pd.read_csv("train.csv")
    data = data.drop(["id"], axis=1)

    data.insert(len(data.columns)-1, 'winter', data['arrival_month'].apply(lambda x: 1 if (x == 1) or (x == 2) or (x == 12) else 0))
    data.insert(len(data.columns)-1, 'spring', data['arrival_month'].apply(lambda x: 1 if (x == 3) or (x == 4) or (x == 5) else 0))
    data.insert(len(data.columns)-1, 'summer', data['arrival_month'].apply(lambda x: 1 if (x == 6) or (x == 7) or (x == 8) else 0))
    data.insert(len(data.columns)-1, '2017', data['arrival_year'].apply(lambda x: 1 if (x == 2017) else 0))
    data.insert(len(data.columns)-1, 'totalPeople', data['no_of_adults'] + data['no_of_children'])
    #data.insert(len(data.columns)-1, 'moreChildren', data['no_of_children'].apply(lambda x: 1 if x > data['no_of_adults'] else 0))
    data = data.drop(['arrival_month'], axis = 1)
    data = data.drop(['arrival_year'], axis = 1)
    print (data.head())
    # print(data.columns)

    pca = PCA(n_components=2)
    pcaData = data.to_numpy()[:, :-1]
    pca.fit(pcaData)
    print(pca.explained_variance_ratio_)
    pcaT = pca.transform(pcaData)
    visualize(pcaT, data.booking_status)

    X_train, X_test, y_train, y_test = train_test_split(pcaData, data.booking_status, test_size=0.2, shuffle=True)
    # print(f"{X_train.shape} : {X_test.shape} : {y_train.shape} : {y_test.shape}")
    scaler = StandardScaler()
    scaler.fit(X_train)
    X_train = scaler.transform(X_train)
    X_test = scaler.transform(X_test)

    #shallow_model(X_train, X_test, y_train, y_test)

    # data.plot(kind ='box',subplots = True,sharex= False,sharey=False,figsize=(15,15))
    # plt.show()

    deepModel(X_train, X_test, y_train, y_test)

In [154]:
def deepModel(X_train, X_test, y_train, y_test):
    model = Sequential()
    model.add(Dense(19, input_shape=(20,), activation='tanh'))
    model.add(Dropout(0.2))
    model.add(Dense(15, activation='tanh'))
    model.add(Dropout(0.2))
    model.add(Dense(10, activation='tanh'))
    model.add(Dropout(0.2))
    model.add(Dense(5, activation='tanh'))
    model.add(Dropout(0.1))
    model.add(Dense(5, activation='tanh'))
    model.add(Dropout(0.05))
    model.add(Dense(1, activation='sigmoid'))

    callback = EarlyStopping(monitor='loss', patience=3)
    # compile the keras model
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

    # fit the keras model on the dataset
    model.fit(X_train, y_train, epochs=200, batch_size=100, callbacks = [callback])

    # evaluate the keras model
    _, accuracy = model.evaluate(X_train, y_train)
    _, accuracyTest = model.evaluate(X_test, y_test)
    print(f'Train Accuracy: {accuracy}')
    print(f'Test Accuracy: {accuracyTest}')

In [119]:
def shallow_model(X_train, X_test, y_train, y_test):
    clf = SVC()
    clf.fit(X_train, y_train)
    predictions = clf.predict(X_test)
    ac = accuracy_score(y_test, predictions)
    train_predict = clf.predict(X_train)
    acTrain = accuracy_score(y_train, train_predict)
    print(f"Train Acc: {acTrain}")
    print(f"Test Acc: {ac}")

In [120]:
def visualize(pcaT, data):
    plt.scatter(pcaT[:, 0], pcaT[:, 1], c=data, cmap='coolwarm', alpha=0.5, s=4)
    plt.show()

In [160]:
if __name__ == "__main__":
    main()

ValueError: The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().